# VSM "ONe time RUn "

In [6]:
# -*- coding: utf-8 -*-
"""VSM.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1T4deHJMx122ybXYUXVTZlwhIbG3bAAYo

## PreProcessing
"""

def tokenization_caseFoldingg() :
    noDoc = 50
    tokens = {}

    for i in range(noDoc) :
        myfile = open('ShortStories\{0}.txt'.format(i+1), encoding='utf-8')
        string  = myfile.read()
        string.strip()
        re.sub('[^A-Za-z0-9]+', '',string)
        dictt = {}
        dictt = invertedIndexx(string.lower(),i+1)
        myfile.close()
        for x,y in dictt.items() :
           if tokens.get(x) == None:
                tokens.update({x:[]})
           tokens[x].extend(y)
    
    return tokens

def StopWordsRemoval(token):
    myfile = open('Stopword-List.txt',encoding='utf-8')
    string = myfile.read()
    dictt = {}
    for x,y in token.items() :
      for yy in y :
        if yy not in string.split() :
            if dictt.get(x) == None :
                dictt.update({x:[]})
            dictt[x].append(yy)
    myfile.close()
    return dictt

def invertedIndexx(string,j): 
    s=''
    tokens = {}
    for i in range(len(string)) :
        if string[i] != ' ' and string[i].isalnum():
            s=s+string[i]
        elif s!= '' :
                
            if tokens.get(j) == None :
                    tokens.update({j:[]})
            tokens[j].append(s.lower())
            s=''
    
    return tokens

import nltk 
def stemming(token) :

  from nltk.stem import PorterStemmer
  porter = PorterStemmer()

  temp = {}
  for docid,words in token.items() :
    for word in words :
      newword = porter.stem(''.join(map(str, word)))
      if temp.get(docid) == None :
                  temp.update({docid:[]})
      
      temp[docid].append(newword)

  

  return temp

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
def lemmatization(token):
 
  
  wordnet_lemmatizer = WordNetLemmatizer()

  # newword = wordnet_lemmatizer.lemmatize("better")
  # print(newword)
  temp = {}
  for docid,words in token.items() :
    for word in words :
      newword = wordnet_lemmatizer.lemmatize(''.join(map(str, word)))
      if temp.get(docid) == None :
                  temp.update({docid:[]})
      
      temp[docid].append(newword)

  

  return temp

def unique(list1,unique_list):
 
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    
    return unique_list

"""## save in DISK"""

def save(invertedIndex,name) :
  import json 
  with open('GUI\{0}.json'.format(name), 'w') as ij:
    json.dump(invertedIndex,ij)

  ij.close();

"""## MAIN (pre and save)"""

def sortIndex(tokens) :
  n = 50
  for i in range(n) :
    tokens[i+1].sort()

  return tokens

import re #regular Expression 

def MainIndex() :
  #breaking string into word and applying caseFolding 
  tokens  = tokenization_caseFoldingg()
  
  invertedIndex = tokens
  print("StopWordsRemoval->")
  invertedIndex = StopWordsRemoval(invertedIndex)
  print("Okey")
  print("lemmatization->")
  invertedIndex = lemmatization(invertedIndex)
  print("Okey")
  #invertedIndex = stemming(invertedIndex)
  invertedIndex = sortIndex(invertedIndex)
  save(invertedIndex,'invertedIndex')

MainIndex()

"""## Reading from Disk"""

import json
with open("GUI\invertedIndex.json", 'r') as ii:
    Inverted_index = json.load(ii)

"""## VECTOR SPACE MODEL

### INDEXING

#### TermFrequency
"""

def calTermFrequency(tokens,uniquelist) :
  n = 50

  vectorSpaceModel =  {}
  
  #InvertedIndex
  for i in range(n) :
 
    for Uword in uniquelist :
      counter = 0 
      counter = tokens[str(i+1)].count(Uword)
      #print("here==",Uword,counter,i)
      if vectorSpaceModel.get(i+1) == None :
        vectorSpaceModel.update({(i+1):[]})
      vectorSpaceModel[i+1].append(counter)

  return vectorSpaceModel

def calTermFrequency_query(queryy,uniquelist) :
  n = 50

  query_tf =  {}
  
  #InvertedIndex

  for Uword in uniquelist :
    counter = 0 
    counter = queryy.count(Uword)
    # if counter >= 1 :
    #   print("yes i am here ")
    #print("here==",Uword,counter,i)
    if query_tf.get('query') == None :
      query_tf.update({('query'):[]})
    query_tf['query'].append(counter)

  return query_tf


"""#### Document Frequency"""

def DocumentFrequency(tokens,uniquelist) :
  df = []
  for j in range(len(uniquelist)) :
     counter = 0 
     for i in range(50):
       if tokens[i+1][j] != 0 :
          counter = counter + 1
      
     df.append(counter)
  
  df = cal_idf(df,50)

  return df

"""#### TF IDF"""

import math 
def cal_idf(df,n) :
  for i in range(len(df)) :
    idf = math.log(df[i],10) / n 
    #idf =  math.log(n/df[i],10)
    df[i] = idf
  return df

def calculateTFIDF(vsm , uniquelt , df) :

  for i in range(len(uniquelt)) :
    
    for j in range(50) :
      vsm[j+1][i] = vsm[j+1][i] * df[i]

  return vsm



"""#### main """

def MainVSM(tokens,uniquelist) :
  
  
  #calculate the Unique list for VSM
  for i in range(50) :
    uniquelist = unique(tokens[str(i+1)],uniquelist)
 
  uniquelist.sort()
  
  save(uniquelist,"PUniqueList")
#   print("calTermFrequency->")
  VSM  = calTermFrequency(tokens,uniquelist)
  print("Okey")
  save(VSM,"TF")
#   print("DocumentFrequency->")
  df = DocumentFrequency(VSM , uniquelist)
#   print("Okey")
#   print("calculateTFIDF->")
  VSM = calculateTFIDF(VSM,uniquelist,df)
#   print("Okey")
  
  save(VSM,"VSM")

  return VSM
#cal


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
StopWordsRemoval->
Okey
lemmatization->
Okey


## Evaluation 

In [12]:
def EVALUATE() :
    myfile = open('test.txt')
    string  = myfile.read()
    listt = list(string.split(" "))
    index_query = 0
    index_result= 0 
    index_len = 0 
    i = 0
    for words in listt :
        i += 1
        if words == 'Query:' :
            index_query = i
        if words == '\nResult:' :
            index_result = i 
        if words == '\nLength:' :
            index_len = i 
    print(index_query,index_result,index_len)
    query_input = []
    result = []
    leng = 0 
    for i in range(index_query,index_result-1) :
        query_input.append( listt[i] )
    for i in range(index_result,index_len-1) :
        result.append( listt[i] )

    leng = listt[index_len] 
    print(query_input)
    print(result)
    print(leng)
    return_DICT = {
        'query':query_input,
        'result':result,
        'len':leng
    }
    return return_DICT


In [13]:
import json
with open("GUI\Result.json", 'r') as ii:
    result = json.load(ii)
print(result)



[6, 11, 12, 16, 18, 22, 28, 41]


# VSM 

In [16]:
uniquelist = []
global_VSM = MainVSM(Inverted_index,uniquelist)

"""## Query"""

import json
with open("GUI\VSM.json", 'r') as ii:
    global_VSM = json.load(ii)


with open("GUI\pUniqueList.json", 'r') as ij:
    uniquelist = json.load(ij)

"""### Preprocessing """

def PreprocessingQuery(queryy) :
#   print("CaseFolding_query->")
  queryy = CaseFolding_query(queryy)
#   print("Okey")
#   print("stopWordsRemoval_query->")
  queryy = stopWordsRemoval_query(queryy)
#   print("Okey")
#   print("lemmatization_query->")
  queryy = lemmatization_query(queryy)
#   print("Okey")
  
  #queryy = stemming_query(queryy)
  return queryy

def CaseFolding_query(query) :
  listt= []
  for words in query :
    listt.append(words.lower())
  return listt



"""#### Lemmatization 

"""



def lemmatization_query(query) :
  wordnet_lemmatizer = WordNetLemmatizer()

  temp = []

  for word in query :
    newword = wordnet_lemmatizer.lemmatize(''.join(map(str, word)))
    temp.append(newword)

  return temp

"""#### stopWordRemoval_query"""

def stopWordsRemoval_query(queryy) :
    myfile = open('Stopword-List.txt',encoding='utf-8')
    string = myfile.read()
    
    #temp variable to store new words
    temp = []
    for word in queryy :
      if word not in string.split() :
        temp.append(word)
    return temp

"""#### stemming_query """

import nltk 
def stemming_query(queryy) :
  from nltk.stem import PorterStemmer
  porter = PorterStemmer()

  temp = []
  for word in queryy :
    newword = porter.stem(''.join(map(str, word)))
    temp.append(newword) 
  return temp



"""### MAIN"""

#input Queries
def Inputt() :
    print("Enter Query")
    querystring = input()
    query= Convert(querystring)
    return query

#========string to list converter=============
def Convert(string):
    li = list(string.split(" "))
    return li

#=================cosine Functionality===========
def magnitude(values) :
  mag = 0
  for i in range(len(values)) :
    mag = mag + math.pow(values[i], 2)
  
  mag = math.sqrt(mag)
  return mag

"""#### DotProduct """

def DotProduct(doc,queryy) :
  dotprod = 0
  for i in range (len(doc)) :
    dotprod = dotprod + doc[i]*queryy[i]

  return dotprod
#Taking input and storing in list
# chalo='y'
# while(chalo=="Y" or chalo=='y') :
query = []
ret_dict = EVALUATE()
query = ret_dict['query']

##PreProcessing the Query 
query = PreprocessingQuery(query)
global_query_TF = calTermFrequency_query(query,uniquelist)
save(global_query_TF ,"query")
save(uniquelist,"uni")
"""#############  MainRANKING  ################## 

"""
# now i have 2 global varible 
# i) global_query_TF
# ii) global_VSM
COS_SIM = {}
#storing the magnitude of query as it will use in all doc
# print("magnitudeQuery->")
mag_query = magnitude(global_query_TF['query'])
# print("Okey")
#calculating for each document 
for i in range(50) :
    # print("DotProduct->")
    numerator = DotProduct(global_VSM[str(i+1)],global_query_TF['query'])
    # print("Okey")
    # print("magnitudeDOC->")
    mag_doc =  magnitude(global_VSM[str(i+1)])
    # print("Okey")
    # print("COSINE_SIM->")
    denominator = mag_doc * mag_query 
    try :
        ans = numerator / denominator
        COS_SIM.update({'SIMDOC{0}'.format(i+1):ans})
        # print("OKEY")
    except :
        ans = 0
        # print("Not FOUND-OKEY")

#SIM
queryResult = []
# print("RANKGING->")
for i in range(len(COS_SIM)) :

    val = COS_SIM['SIMDOC{0}'.format(i+1)]
    if val > 0.005:
        #print(i+1,val)
        queryResult.append(i+1)
save(queryResult,"Result")
print(queryResult)
flag = True 
for val_res in ret_dict['result'] :
    if int(val_res) in queryResult :
        flag == True
    else :
        flag = False 
if flag :
    print("Corret")
else :
    print("wrong")
#   print("\n\twant to continue?\t Y/y or N/n")
#   chalo = input()
  


Okey
1 4 8
['across', 'adventures']
['9', '20', '44']
3
[9, 20, 44]
Corret
